In [1]:
!pip install -q -U watermark

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os

In [3]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

Author: Leonardo da Silva Neves



In [4]:
# Carregando DataFrames das edições do censo da educação superior.
censo_23 = 'censup/MICRODADOS_CADASTRO_CURSOS_2023.CSV'
censo_22 = 'censup/MICRODADOS_CADASTRO_CURSOS_2022.CSV'
censo_21 = 'censup/MICRODADOS_CADASTRO_CURSOS_2021.CSV'
censo_20 = 'censup/MICRODADOS_CADASTRO_CURSOS_2020.CSV' 

In [5]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_20, 'rb') as f1:
    raw_data_23 = f1.read(1000000)

In [6]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data_23)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

codificação detectada: ISO-8859-1 (confiança: 0.73%)


In [7]:
# Carregando o DataFrame em uma variável.
df_23 = pd.read_csv(censo_23, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [8]:
df_22 = pd.read_csv(censo_22, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [9]:
df_21 = pd.read_csv(censo_21, sep=';', encoding=cod_23, low_memory=False, decimal=',')

In [10]:
df_20 = pd.read_csv(censo_20, sep=';', encoding=cod_23, low_memory=False, decimal=',')

### Identificando as colunas que são comuns a todos os DataFrames e, em seguida, manter apenas essas colunas em cada um deles.
Este procedimento garante que, ao concatenar, você não serão mostradas colunas com valores 'NaN'.

In [11]:
# 1.1 Encontrando as colunas comuns
# Carregando os dfs em uma lista
dfs = [df_20, df_21, df_22, df_23]

In [12]:
# 1.2 Pegando convertendo a lista de colunas de cada df para o tipo set()
colunas_20 = set(dfs[0].columns)
colunas_21 = set(dfs[1].columns)
colunas_22 = set(dfs[2].columns)
colunas_23 = set(dfs[3].columns)

In [13]:
# Encontra a interseção de todas as colunas
# O resultado será um conjunto de colunas presentes em todos os DataFrames
colunas_comuns = colunas_20.intersection(colunas_21).intersection(colunas_22).intersection(colunas_23)

In [14]:
# Converte o conjunto de volta para o formato de lista
colunas_comuns_lista = list(colunas_comuns)

In [15]:
print(f"Número total de colunas em 2020: {len(colunas_20)}")
print(f"Número total de colunas em 2021: {len(colunas_21)}")
print(f"Número total de colunas em 2022: {len(colunas_22)}")
print(f"Número total de colunas em 2023: {len(colunas_23)}")
print(f"Número de colunas comuns {len(colunas_comuns)}")

Número total de colunas em 2020: 200
Número total de colunas em 2021: 200
Número total de colunas em 2022: 200
Número total de colunas em 2023: 202
Número de colunas comuns 199


In [16]:
# Iterando sobre a lista de DataFrames
dfs_padronizados = []
for df_ano in dfs:
    # Selecionar apenas as colunas comuns para cada DataFrame
    df_padronizado = df_ano[colunas_comuns_lista]
    dfs_padronizados.append(df_padronizado)

# Concatenando os DataFrames padronizados em um único DataFrame final
df_completo_padronizado = pd.concat(dfs_padronizados, ignore_index=True)

In [17]:
df_completo_padronizado.shape

(2034106, 199)

In [18]:
# Criando um df apenas com os cursos da modalidade EaD
df_ead_geral = df_completo_padronizado[df_completo_padronizado['TP_MODALIDADE_ENSINO'] == 2].copy()

In [19]:
df_ead_geral.shape

(1890415, 199)

In [20]:
# Substuindo valores da variável 'TP_MODALIDADE_ENSINO'
df_ead_geral['TP_MODALIDADE_ENSINO'] = df_ead_geral['TP_MODALIDADE_ENSINO'].replace(2, 'EaD')

In [21]:
# Alterando o parâmetro máximo de visualizações de colunas
pd.set_option('display.max_columns', 202)

In [22]:
df_ead_geral['NU_ANO_CENSO'] = pd.to_datetime(df_ead_geral['NU_ANO_CENSO'], format='%Y')

In [23]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_ead_geral.columns if col.startswith('QT_')]

In [24]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
df_ead_geral[colunas_quantidade] = df_ead_geral[colunas_quantidade].fillna(0)

In [25]:
df_ead_geral['NO_CINE_AREA_GERAL'].value_counts()

NO_CINE_AREA_GERAL
Negócios, administração e direito                             607300
Educação                                                      427249
Computação e Tecnologias da Informação e Comunicação (TIC)    178487
Saúde e bem-estar                                             172779
Engenharia, produção e construção                             152182
Serviços                                                      133410
Artes e humanidades                                           101984
Ciências sociais, comunicação e informação                     71919
Ciências naturais, matemática e estatística                    25795
Agricultura, silvicultura, pesca e veterinária                 16814
Programas básicos                                               2496
Name: count, dtype: int64

In [26]:
df_ead_geral['NO_CINE_AREA_ESPECIFICA'].value_counts()

NO_CINE_AREA_ESPECIFICA
Negócios e administração                                                                                     577085
Educação                                                                                                     423739
Computação e Tecnologias da Informação e Comunicação (TIC)                                                   177014
Saúde                                                                                                        124622
Engenharia e profissões correlatas                                                                            89551
Artes                                                                                                         63918
Serviços de segurança                                                                                         58729
Serviços pessoais                                                                                             53888
Bem-estar                                       

In [37]:
df_ead_geral['NO_CINE_AREA_DETALHADA'].value_counts()

NO_CINE_AREA_DETALHADA
Gestão e administração                                                     346705
Formação de professores em áreas específicas (exceto Letras)               246567
Formação de professores de letras                                           81047
Formação de professores sem áreas específicas                               79832
Marketing e propaganda                                                      74876
                                                                            ...  
Programas interdisciplinares abrangendo serviços                              382
Programas abrangendo educação em processo de definição da classificação       136
Medicina                                                                       67
Mineração e extração                                                           42
Engenharia e profissões correlatas sem definição precisa                       41
Name: count, Length: 76, dtype: int64

In [31]:
df_ead_geral.sample(5)

,QT_INSCRITO_TOTAL_EAD,QT_ING_SELECAO_SIMPLIFICA,NU_ANO_CENSO,QT_ING_25_29,QT_MAT,QT_ING_PROUNII,QT_CONC_30_34,QT_ING_RVSOCIAL_RF,QT_ING_18_24,QT_ING_PRETA,QT_ING_AVALIACAO_SERIADA,QT_MAT_CORND,QT_VG_TOTAL_NOTURNO,QT_CONC_18_24,QT_CONC_AMARELA,QT_CONC_NACESTRANG,QT_MOB_ACADEMICA,QT_ING_PARDA,QT_ING_ENEM,QT_ALUNO_DEFICIENTE,QT_MAT_PROUNIP,CO_MUNICIPIO,CO_CURSO,QT_ING_35_39,TP_ORGANIZACAO_ACADEMICA,QT_ING,QT_MAT_NOTURNO,QT_CONC_0_17,QT_CONC_60_MAIS,QT_INSC_VG_REMANESC,QT_ING_MOB_ACADEMICA,QT_ING_50_59,QT_CONC_RPFIES,QT_ING_VG_PROG_ESPECIAL,QT_MAT_APOIO_SOCIAL,QT_CONC_FEM,QT_INSC_PROC_SELETIVO,QT_CONC_PARDA,QT_CONC_APOIO_SOCIAL,QT_MAT_0_17,QT_ING_OUTRO_TIPO_SELECAO,TP_CATEGORIA_ADMINISTRATIVA,QT_CONC_NACBRAS,QT_ING_RVPDEF,CO_CINE_AREA_DETALHADA,QT_ING_60_MAIS,QT_CONC_CORND,QT_ING_40_49,QT_SIT_DESVINCULADO,QT_ING_PROCNAOINFORMADA,QT_CONC_DIURNO,CO_CINE_AREA_GERAL,QT_CONC_NRPFIES,QT_CONC_RVOUTROS,CO_REGIAO,CO_UF,QT_MAT_35_39,QT_MAT_MASC,QT_CONC_PROCESCPRIVADA,QT_INSC_VG_NOVA,QT_CONC,QT_ING_NOTURNO,CO_IES,NO_CINE_AREA_DETALHADA,QT_MAT_RVPDEF,QT_CONC_35_39,QT_MAT_25_29,QT_ING_PARFOR,QT_MAT_NACBRAS,QT_INSCRITO_TOTAL_NOTURNO,QT_ING_FINANC_REEMB_OUTROS,QT_CONC_PROUNIP,QT_ING_VG_REMANESC,QT_MAT_PROCNAOINFORMADA,QT_MAT_40_49,QT_ING_FINANC_NREEMB,TP_MODALIDADE_ENSINO,QT_ING_30_34,QT_MAT_FEM,QT_VG_PROG_ESPECIAL,QT_CONC_NOTURNO,QT_MAT_INDIGENA,QT_MAT_PRETA,QT_ING_DIURNO,QT_CONC_BRANCA,QT_CONC_PROCNAOINFORMADA,QT_CONC_25_29,NO_CINE_ROTULO,QT_CONC_FINANC,QT_ING_APOIO_SOCIAL,CO_CINE_AREA_ESPECIFICA,QT_MAT_NACESTRANG,TP_GRAU_ACADEMICO,QT_CURSO,QT_ING_FEM,QT_MAT_FINANC_REEMB,QT_CONC_FINANC_NREEMB_OUTROS,QT_CONC_ATIV_EXTRACURRICULAR,QT_ING_FINANC_REEMB,QT_MAT_FINANC_NREEMB_OUTROS,QT_MAT_ATIV_EXTRACURRICULAR,QT_ING_FINANC,QT_MAT_50_59,QT_MAT_PARDA,QT_ING_NRPFIES,QT_ING_FINANC_NREEMB_OUTROS,QT_ING_AMARELA,QT_ING_NACESTRANG,QT_CONC_FIES,QT_MAT_RESERVA_VAGA,QT_APOIO_SOCIAL,QT_MAT_RPFIES,QT_MAT_AMARELA,QT_MAT_RVSOCIAL_RF,IN_GRATUITO,QT_ING_CORND,QT_VG_TOTAL,QT_MAT_FIES,QT_CONC_RESERVA_VAGA,QT_ING_FIES,QT_ING_VESTIBULAR,QT_VG_NOVA,QT_ING_INDIGENA,QT_CONC_RVPDEF,QT_MAT_FINANC_REEMB_OUTROS,QT_CONC_FINANC_REEMB_OUTROS,QT_MAT_FINANC,TP_REDE,QT_CONC_PRETA,QT_MAT_NRPFIES,QT_CONC_INDIGENA,QT_INSC_VG_PROG_ESPECIAL,QT_CONC_PROCESCPUBLICA,QT_ING_RVETNICO,QT_ING_RVOUTROS,QT_CONC_MASC,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE,QT_CONC_RVSOCIAL_RF,NO_CINE_AREA_ESPECIFICA,QT_ING_DEFICIENTE,QT_ATIV_EXTRACURRICULAR,QT_ING_PROCESCPUBLICA,QT_PARFOR,QT_ING_RESERVA_VAGA,QT_VG_REMANESC,TP_DIMENSAO,QT_CONC_RVETNICO,QT_VG_PROC_SELETIVO,QT_SIT_TRANCADA,SG_UF,QT_ING_PROUNIP,QT_ING_MASC,QT_MAT_DIURNO,QT_MAT_30_34,QT_CONC_PROUNII,QT_ING_EGR,NO_REGIAO,QT_MAT_RVOUTROS,QT_ING_RPFIES,QT_ING_BRANCA,QT_MAT_MOB_ACADEMICA,NO_CURSO,QT_SIT_TRANSFERIDO,QT_CONC_FINANC_NREEMB,QT_CONC_RVREDEPUBLICA,QT_MAT_PROCESCPUBLICA,QT_ING_PROCESCPRIVADA,TP_NIVEL_ACADEMICO,QT_MAT_FINANC_NREEMB,QT_ING_0_17,NO_UF,NO_CINE_AREA_GERAL,QT_INSCRITO_TOTAL_DIURNO,QT_ING_ATIV_EXTRACURRICULAR,NO_MUNICIPIO,QT_CONC_PARFOR,QT_ING_OUTRA_FORMA,QT_CONC_50_59,QT_ING_RVREDEPUBLICA,QT_ING_VG_NOVA,QT_MAT_PROCESCPRIVADA,QT_VG_TOTAL_EAD,QT_CONC_MOB_ACADEMICA,IN_CAPITAL,QT_MAT_RVETNICO,QT_SIT_FALECIDO,QT_MAT_BRANCA,QT_ING_NACBRAS,QT_VG_TOTAL_DIURNO,QT_CONC_40_49,QT_MAT_60_MAIS,QT_MAT_PROUNII,QT_MAT_RVREDEPUBLICA,QT_INSCRITO_TOTAL,QT_MAT_PARFOR,QT_MAT_18_24,QT_ING_PROC_SELETIVO,QT_CONC_FINANC_REEMB
941854,0,0,2022-01-01,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2613602.0,1467671,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,212,0,0,0,3,0,0,2,0,0,2.0,26.0,0,0,0,0,0,0,2835,"Moda, design de interiores e desenho industrial",0,0,0,0,0,0,0,0,0,0,0,0,EaD,0,0,0,0,0,0,0,0,0,0,Design de interiores,0,0,21,0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,Artes,0,0,0,0,0,0,2,0,0,0,PE,0,1,0,0,0,0,Nordeste,0,0,1,0,Design De Interiores,0,0,0,0,1,1,0,0,Pernambuco,Artes e humanidades,0,0,São José do Egito,0,0,0,0,1,0,0,0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1725441,0,0,2023-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,33034

### 1. Métricas de Conclusão e Evasão
Métricas importantes para entender a eficiência e retenção dos cursos.

In [34]:
df_ead_geral['QT_ING'].sum()

10901311

In [35]:
df_ead_geral.groupby('NU_ANO_CENSO')['QT_ING'].sum().sort_values(ascending=False)

NU_ANO_CENSO
2023-01-01    3314402
2022-01-01    3100556
2021-01-01    2477374
2020-01-01    2008979
Name: QT_ING, dtype: int64

In [37]:
df_ead_geral.groupby(['NU_ANO_CENSO', 'NO_REGIAO'])['QT_ING_PRETA'].sum().sort_values(ascending=False)

NU_ANO_CENSO  NO_REGIAO   
2023-01-01    Sudeste         128572
2022-01-01    Sudeste         109540
2021-01-01    Sudeste          77013
2020-01-01    Sudeste          62882
2023-01-01    Nordeste         40171
2022-01-01    Nordeste         37051
2021-01-01    Nordeste         26876
2020-01-01    Nordeste         25095
2023-01-01    Sul              24055
2022-01-01    Sul              23095
2021-01-01    Sul              19575
2023-01-01    Centro-Oeste     18485
2022-01-01    Centro-Oeste     18015
2020-01-01    Sul              15130
2021-01-01    Centro-Oeste     14115
2022-01-01    Norte            11204
2023-01-01    Norte            10853
2020-01-01    Centro-Oeste     10704
2021-01-01    Norte             8612
2020-01-01    Norte             7372
Name: QT_ING_PRETA, dtype: int64

In [41]:
df_ead_geral['TP_GRAU_ACADEMICO'].isna().sum()

1417

In [42]:
df_ead_geral['TP_GRAU_ACADEMICO'].value_counts()

TP_GRAU_ACADEMICO
3.0    919145
1.0    561054
2.0    407716
0.0      1083
Name: count, dtype: int64

         QT_ING_VESTIBULAR  QT_ING_VG_PROG_ESPECIAL  QT_MAT_PARDA  \
428                      0                        0             0   
429                      0                        0             0   
1578                     0                        0             0   
5055                     0                        0             0   
181095                   0                        0             0   
...                    ...                      ...           ...   
2031120                  2                        0             0   
2032085                 11                        0             2   
2032853                  0                        0             0   
2032957                  4                        0             1   
2033460                  0                        0             0   

         QT_ING_VG_REMANESC  QT_MAT_MOB_ACADEMICA  QT_PARFOR  \
428                       0                     0          0   
429                       0                